In [1]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
import torch

query = "Give me a short introduction to large language model."

# model_name = "Qwen/Qwen3-4B-Instruct-2507"
# model_name = "Qwen/Qwen3-4B" 
model_name = "Qwen/Qwen3-0.6B"

bnb_config = BitsAndBytesConfig( # quantitazation (q in qlora) part
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_use_double_quant=True,
  bnb_4bit_compute_dtype=torch.bfloat16,
)

qwen = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config=bnb_config,
  device_map="auto",
)

model = PeftModel.from_pretrained(qwen, "../out/checkpoint-114")

tokenizer = AutoTokenizer.from_pretrained(model_name)

model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 1024)
        (layers): ModuleList(
          (0-27): 28 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1024, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.

In [2]:
def tokenize(tokenizer, prompt, thinking = False):
  messages = [
    {"role": "user", "content": prompt}
  ]
  text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=thinking
  )
  return tokenizer([text], return_tensors="pt").to(model.device)

def generate(model, tokenizer, prompt, thinking = False):
  tokens = tokenize(tokenizer, prompt, thinking)
  generated_ids = model.generate(
    **tokens,
    max_new_tokens=32768
  )
  output_ids = generated_ids[0][len(tokens.input_ids[0]):].tolist() 

  # parsing thinking content
  try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
  except ValueError:
    index = 0
  
  return {
    "thinking_content" : tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n"),
    "content" : tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
  }

model.eval()
prompt = "Give me a short introduction to large language model."
content = generate(model, tokenizer, prompt, False)
{
  "prompt" :prompt,
  "output" : content["content"]
}

{'prompt': 'Give me a short introduction to large language model.',
 'output': 'A large language model (LLM) is a machine learning system designed to understand and generate human-like text in natural conversations, including writing, reading, and other natural language tasks. These models are trained on massive datasets and are capable of understanding context, sarcasm, and cultural nuances. They are used for a wide range of tasks, from content creation to language translation and information retrieval.'}